In [1]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalizationV2

In [2]:
train_path = '../archive/seg_train/'
test_path = '../../black_box/data/black_box_single/seg_test/'

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   horizontal_flip=True,
                                   shear_range=0.2,
                                   zoom_range=0.2)
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (64, 64),
                                                 batch_size=64,
                                                 class_mode = 'categorical')

Found 14034 images belonging to 6 classes.


In [4]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (64, 64),
                                            batch_size=64,
                                            class_mode = 'categorical')

Found 3000 images belonging to 6 classes.


In [5]:
cnn = Sequential()
cnn

Metal device set to: Apple M1


2022-08-29 21:36:01.937919: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-29 21:36:01.938045: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
cnn.add(Convolution2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Convolution2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Convolution2D(filters=128, kernel_size=3, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Convolution2D(filters=256, kernel_size=3, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Dropout(0.4)),

cnn.add(Flatten())

cnn.add(Dense(units=256, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(Dropout(0.25))

cnn.add(Dense(units=128, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(Dropout(0.25))

cnn.add(Dense(units=64, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(Dropout(0.25))

cnn.add(Dense(units=24, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(Dropout(0.25))

cnn.add(Dense(units=6, activation='softmax'))

In [7]:
cnn.compile(optimizer = 'sgd', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [8]:
%%time
cnn.fit(training_set, steps_per_epoch=len(training_set), validation_data = test_set, validation_steps=len(test_set), epochs = 10)

Epoch 1/10


2022-08-29 21:36:02.590110: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-29 21:36:03.690355: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


220/220 [==============================] - ETA: 0s - loss: 1.6309 - accuracy: 0.3826

2022-08-29 21:36:35.738202: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


220/220 [==============================] - 36s 153ms/step - loss: 1.6309 - accuracy: 0.3826 - val_loss: 1.7905 - val_accuracy: 0.2880
Epoch 2/10
220/220 [==============================] - 28s 125ms/step - loss: 1.2087 - accuracy: 0.5217 - val_loss: 1.2071 - val_accuracy: 0.5227
Epoch 3/10
220/220 [==============================] - 31s 139ms/step - loss: 1.0816 - accuracy: 0.5800 - val_loss: 1.2337 - val_accuracy: 0.5053
Epoch 4/10
220/220 [==============================] - 41s 184ms/step - loss: 0.9877 - accuracy: 0.6256 - val_loss: 1.2693 - val_accuracy: 0.5050
Epoch 5/10
220/220 [==============================] - 35s 160ms/step - loss: 0.9184 - accuracy: 0.6536 - val_loss: 1.1935 - val_accuracy: 0.5420
Epoch 6/10
220/220 [==============================] - 37s 167ms/step - loss: 0.8811 - accuracy: 0.6776 - val_loss: 1.3038 - val_accuracy: 0.5043
Epoch 7/10
220/220 [==============================] - 42s 193ms/step - loss: 0.8241 - accuracy: 0.7007 - val_loss: 0.9796 - val_accuracy: 0.6

In [9]:
accuracy=cnn.evaluate(test_set, steps=len(test_set),verbose=2)
print('Accuracy of the model on the test set: ',accuracy[1])

47/47 - 2s - loss: 0.9670 - accuracy: 0.6367 - 2s/epoch - 47ms/step
Accuracy of the model on the test set:  0.6366666555404663
